In [ ]:
TF_RECORD_FILE = '/media/commaai-03/Data/dataset/segment-1208303279778032257_1360_000_1380_000_with_camera_labels.tfrecord'

In [ ]:
import os
import math
import tensorflow as tf
import numpy as np

In [ ]:
# For running session without tensor running.
tf.enable_eager_execution()

# Waymo open dataset Modules
from waymo_open_dataset import dataset_pb2 as open_dataset
from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import frame_utils

In [ ]:
dataset = tf.data.TFRecordDataset(filenames=TF_RECORD_FILE, 
                                  compression_type='')
for data in dataset:
    frame = open_dataset.Frame()
    frame.ParseFromString(bytearray(data.numpy()))
    break
    
(range_images, camera_projections, range_image_top_pose) = \
    frame_utils.parse_range_image_and_camera_projection(frame)

In [ ]:
?range_image_top_pose

In [ ]:
print(frame.context)

#### Visualize camera Images and Labels

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline

In [ ]:
def show_image_with_boxes(camera_image, camera_labels, layout, cmap=None):
    '''
    Show a camera image and the given camera labels.
    Args:
    - camera_image:
    - camera_labels:
    - layout:
    - cmap:
    '''
    
    axs = plt.subplot(*layout)
    
    # Draw the camera labels.
    for camera_labels in frame.camera_labels:
        # Ignore tcamera labels that do not correspond to this camera.
        if camera_labels.name != camera_image.name:
            continue
        
        # Iterate over the individual labels.
        for label in camera_labels.labels:
            # Draw the object bounding box.
            x = label.box.center_x - 0.5 * label.box.length
            y = label.box.center_y - 0.5 * label.box.width
            axs.add_patch(patches.Rectangle(
                xy=(x, y),
                width=label.box.length,
                height=label.box.width,
                linewidth=1, 
                edgecolor='green', facecolor='none'))
            
    # Show the camera image
    plt.imshow(tf.image.decode_image(camera_image.image), cmap=cmap)
    plt.title(open_dataset.CameraName.Name.Name(camera_image.name))
    plt.grid(False)
    plt.axis('off')

In [ ]:
plt.figure(figsize=(25, 20))

for index, image in enumerate(frame.images):
    show_image_with_boxes(image, frame.camera_labels, [3, 3, index+1])

#### Visualize range Images

In [ ]:
plt.figure(figsize=(33,33))

def plot_range_image_helper(data, name, layout, vmin=0, vmax=1, cmap='gray'):
    ''' Plots range image.
    Args:
    - data: range image data
    - name: the image title
    - layout: pyplot layout
    - vmin: minimum value of the passed data
    - vmax: maximum value of the passed data
    - cmap: color map
    '''
    
    plt.subplot(*layout)
    plt.imshow(data, cmap=cmap, vmin=vmin, vmax=vmax)
    plt.title(name)
    plt.grid(False)
    plt.axis('off')
    
def get_range_image(laser_name, index):
    return range_images[laser_name][index]

def show_range_image(range_image, layout_index_start=1):
    ''' Shows range image.
    Args:
    - range_image: the range image data from a given lidar of type MatrixFloat.
    - layout_index_start: layout offset.
    '''
    
    range_image_tensor = tf.convert_to_tensor(range_image.data)
    range_image_tensor = tf.reshape(range_image_tensor, 
                                    range_image.shape.dims)
    lidar_image_mask = tf.greater_equal(range_image_tensor, 0)
    range_image_tensor = tf.where(lidar_image_mask, 
                                  range_image_tensor, 
                                  tf.ones_like(range_image_tensor) * 1e10)
    range_image_range = range_image_tensor[...,0] 
    range_image_intensity = range_image_tensor[...,1]
    range_image_elongation = range_image_tensor[...,2]
    plot_range_image_helper(range_image_range.numpy(), 'range',
                   [8, 1, layout_index_start], vmax=75, cmap='gray')
    plot_range_image_helper(range_image_intensity.numpy(), 'intensity',
                   [8, 1, layout_index_start + 1], vmax=1.5, cmap='gray')
    plot_range_image_helper(range_image_elongation.numpy(), 'elongation',
                   [8, 1, layout_index_start + 2], vmax=1.5, cmap='gray')
    
frame.lasers.sort(key=lambda laser: laser.name)
show_range_image(get_range_image(open_dataset.LaserName.TOP, 0), 1)
#show_range_image(get_range_image(open_dataset.LaserName.TOP, 1), 4)

#### Point Cloud conversion and visualization

In [ ]:
points, cp_points = frame_utils.convert_range_image_to_point_cloud(
    frame, 
    range_images, 
    camera_projections, 
    range_image_top_pose)

points_ri2, cp_points_ri2 = frame_utils.convert_range_image_to_point_cloud(
    frame,
    range_images,
    camera_projections,
    range_image_top_pose,
    ri_index=1)

# 3d points in vehicle frame.
points_all = np.concatenate(points, axis=0)
points_all_ri2 = np.concatenate(points_ri2, axis=0)
# camera projection corresponding to each point.
cp_points_all = np.concatenate(cp_points, axis=0)
cp_points_all_ri2 = np.concatenate(cp_points_ri2, axis=0)

#### Examine number of points in each lidar sensor.

In [ ]:
print(points_all.shape)
print(cp_points_all.shape)
print()
for i in range(5):
    print(points[i].shape)
    print(cp_points[i].shape)

In [ ]:
print(points_all_ri2.shape)
print(cp_points_all_ri2.shape)
print(points_all_ri2[0:2])

print()

for i in range(5):
  print(points_ri2[i].shape)
  print(cp_points_ri2[i].shape)

In [ ]:
?frame_utils.convert_range_image_to_point_cloud